# Données, approches fonctionnelles - énoncé - blaze - odo

Ce notebook illustre le module [Blaze](http://blaze.pydata.org/en/latest/).

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pyensae
from pyquickhelper.helpgen import NbImage
from jyquickhelper import add_notebook_menu
add_notebook_menu()

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from actuariat_python.data import table_mortalite_euro_stat 
table_mortalite_euro_stat()
import pandas
df = pandas.read_csv("mortalite.txt", sep="\t", encoding="utf8", low_memory=False)
df.head()

,annee,valeur,age,age_num,indicateur,genre,pays
0,2015,0.00043,Y01,1.0,DEATHRATE,F,AL
1,2014,0.00043,Y01,1.0,DEATHRATE,F,AL
2,2015,0.00035,Y01,1.0,DEATHRATE,F,AM
3,2009,0.00080,Y01,1.0,DEATHRATE,F,AM
4,2008,0.00067,Y01,1.0,DEATHRATE,F,AM


### Blaze : interface commune

[Blaze](http://blaze.pydata.org/en/latest/) fournit une interface commune, proche de celle des Dataframe, pour de nombreux modules comme [bcolz](http://bcolz.blosc.org/)...

* [Pandas to Blaze](http://blaze.pydata.org/en/latest/rosetta-pandas.html)

In [ ]:
from blaze import Data

c:\Python36_x64\lib\site-packages\odo\backends\pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
c:\Python36_x64\lib\site-packages\blaze\server\server.py:17: ExtDeprecationWarning: Importing flask.ext.cors is deprecated, use flask_cors instead.
  from flask.ext.cors import cross_origin


In [ ]:
bs = Data(df)

In [ ]:
bs.shape

(2866533,)

In [ ]:
bs.schema

dshape("""{
  annee: int64,
  valeur: float64,
  age: ?string,
  age_num: float64,
  indicateur: ?string,
  genre: ?string,
  pays: ?string
  }""")

In [ ]:
life = bs[bs.indicateur == 'LIFEXP']
life.head()

,annee,valeur,age,age_num,indicateur,genre,pays
411000,2015,79.2,Y01,1.0,LIFEXP,F,AL
411001,2014,79.9,Y01,1.0,LIFEXP,F,AL
411002,2015,77.9,Y01,1.0,LIFEXP,F,AM
411003,2009,76.5,Y01,1.0,LIFEXP,F,AM
411004,2008,76.4,Y01,1.0,LIFEXP,F,AM
411005,2007,76.5,Y01,1.0,LIFEXP,F,AM
411006,2006,75.9,Y01,1.0,LIFEXP,F,AM
411007,2015,83.0,Y01,1.0,LIFEXP,F,AT
411008,2014,83.3,Y01,1.0,LIFEXP,F,AT
411009,2013,83.0,Y01,1.0,LIFEXP,F,AT


Le design de Blaze est différent, un filtrage de la base de données retourne un type différent.

In [ ]:
type(bs[bs.indicateur == 'LIFEXP']), type(bs)

(blaze.expr.expressions.Selection, blaze.interactive._Data)

Contrairement à pandas :

In [ ]:
type(df[df.indicateur=='LIFEXP']), type(df)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

### Odo, conversion en tout genre

[odo](https://odo.readthedocs.org/en/latest/) convertit à peu près n'importe quoi en n'importe quoi, fichiers, fichiers compressés, bases de données, Spark, Hadoop...

* [What sorts of URI’s does odo support?](https://odo.readthedocs.org/en/latest/uri.html#what-sorts-of-uri-s-does-odo-support)

In [ ]:
from odo import odo

On supprime le fichier sinon les données s'ajoutent à ce qui est déjà présent dans le fichier existant.

In [ ]:
import os
if os.path.exists("mortalite_compresse.csv.gz"):
    os.remove("mortalite_compresse.csv.gz")

In [ ]:
odo(df, "mortalite_compresse.csv.gz")

Et la relecture :

In [ ]:
try:
    df_lu = odo("mortalite_compresse.csv.gz", pandas.DataFrame)
except AttributeError as e:
    # Erreur découverte : 2 octobre 2017
    print("Error:", e)
    print("On utilise pandas directement.")
    df_lu = pandas.read_csv("mortalite_compresse.csv.gz")
df.shape, df_lu.shape

Error: 'DiGraph' object has no attribute 'edge'
On utilise pandas directement.


((2866533, 7), (2866533, 7))